System found in **Perturbative Diagonalization for Time-Dependent Strong Interactions**: 10.1103/PhysRevApplied.18.024009

In [1]:
# Objects for defining the Hamiltonian
from sympt import *
import sympy as sp

In [2]:
omega_q = RDSymbol('omega_q', real=True, positive=True)
omega_a = RDSymbol('omega_a', real=True, positive=True)

abs_g_p = RDSymbol('|{g_p}|', real=True, positive=True, order=1)
phi_p = RDSymbol('phi_p', real=True)
gp = abs_g_p * sp.exp(-sp.I * phi_p)
omega_p = RDSymbol('omega_p', real=True, positive=True)

Spin = RDBasis('sigma', 2)
s0, sx, sy, sz = Spin.basis

a = BosonOp('a')
ad = Dagger(a)

H0 = - omega_q * sz / 2 + omega_a * (ad * a + sp.Rational(1, 2))
V1 = (gp * sp.exp(-sp.I * omega_p * t) + sp.conjugate(gp) * sp.exp(sp.I * omega_p * t)) * (a + ad) * sx


display_dict({
    sp.Symbol('H_0'): H0,
    sp.Symbol('V_1'): V1,
})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
Eff_Frame = EffectiveFrame(H=H0, V=V1, subspaces=[Spin], verbose = False)

# Standard Schrieffer-Wolff transformation

In [4]:
Eff_Frame.solve(max_order=2, method="SW")
H_eff = Eff_Frame.get_H()

In [5]:
# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_Frame.corrections.values()]

# Reconstructing results obtained in the paper mentioned in top cell

In [6]:
# variables found in the paper

xi_plus = gp / (omega_q + omega_a - omega_p) * sp.exp(-I* omega_p * t) + sp.conjugate(gp) / (omega_q + omega_a + omega_p) * sp.exp(I* omega_p * t)
xi_minus = gp / (omega_q - omega_a + omega_p) * sp.exp(-I* omega_p * t) + sp.conjugate(gp) / (omega_q - omega_a - omega_p) * sp.exp(I* omega_p * t)

gt = gp * sp.exp(-I* omega_p * t) + sp.conjugate(gp) * sp.exp(I* omega_p * t)

### Identity

In [7]:
omega_plus = sp.Symbol('omega_+', real=True, positive=True)
omega_minus = sp.Symbol('omega_-', real=True, positive=True)

# Term obtained in the paper
sp.expand_complex((sp.Rational(1,2) * gt * (sp.re(xi_minus) - sp.re(xi_plus))).expand()).trigsimp().subs({
    omega_q + omega_a : omega_plus,
    omega_q - omega_a : omega_minus,
    hbar:1
}).subs(omega_q**2 - omega_a**2, (omega_plus * omega_minus)).subs({
    omega_a : sp.Rational(1,2) * (omega_plus - omega_minus),
    omega_q : sp.Rational(1,2) * (omega_plus + omega_minus),
    }).expand().cancel().factor()

2*|{g_p}|**2*(omega_p**2 + omega_+*omega_-)*(omega_+ - omega_-)*cos(omega_p*t + phi_p)**2/((-omega_p + omega_+)*(-omega_p + omega_-)*(omega_p + omega_+)*(omega_p + omega_-))

In [11]:
# Term obtained with SymPT
sp.expand_complex(sp.factor_terms(group_by_operators(H2)[1]).expand().factor()).trigsimp().subs(hbar, 1).subs(sp.cos((2*(omega_p*t + phi_p)).factor()) + 1, 2*sp.cos(omega_p*t + phi_p)**2).subs({
    omega_q + omega_a : omega_plus,
    omega_q - omega_a : omega_minus,
    hbar:1
}).subs(omega_q**2 - omega_a**2, (omega_plus*omega_minus)).subs({
    omega_a : sp.Rational(1,2) * (omega_plus - omega_minus),
    omega_q : sp.Rational(1,2) * (omega_plus + omega_minus),
    }).expand().cancel().factor()

2*|{g_p}|**2*(omega_p**2 + omega_+*omega_-)*(omega_+ - omega_-)*cos(omega_p*t + phi_p)**2/((-omega_p + omega_+)*(-omega_p + omega_-)*(omega_p + omega_+)*(omega_p + omega_-))

### $\sigma_z a^\dagger a$

In [12]:
# Term obtained in the paper
sp.expand_complex(-gt * (sp.re(xi_minus) + sp.re(xi_plus))).expand().trigsimp().expand().factor().trigsimp().subs(hbar, 1).subs(sp.cos((2*(omega_p*t + phi_p)).factor()) + 1, 2*sp.cos(omega_p*t + phi_p)**2).subs({
    omega_q + omega_a : omega_plus,
    omega_q - omega_a : omega_minus,
    hbar:1
}).subs(omega_q**2 - omega_a**2, (omega_plus*omega_minus)).subs({
    omega_a : sp.Rational(1,2) * (omega_plus - omega_minus),
    omega_q : sp.Rational(1,2) * (omega_plus + omega_minus),
    }).expand().cancel().factor()

-4*|{g_p}|**2*(-omega_p**2 + omega_+*omega_-)*(omega_+ + omega_-)*cos(omega_p*t + phi_p)**2/((-omega_p + omega_+)*(-omega_p + omega_-)*(omega_p + omega_+)*(omega_p + omega_-))

In [13]:
# Term obtained with SymPT
sp.expand_complex(sp.factor_terms(group_by_operators(H2)[sz*ad*a])).expand().factor().trigsimp().subs(hbar, 1).subs({
    omega_q + omega_a : omega_plus,
    omega_q - omega_a : omega_minus,
    hbar:1
}).subs(omega_q**2 - omega_a**2, (omega_plus*omega_minus)).subs({
    omega_a : sp.Rational(1,2) * (omega_plus - omega_minus),
    omega_q : sp.Rational(1,2) * (omega_plus + omega_minus),
    }).expand().cancel().factor()

-2*|{g_p}|**2*(-omega_p**2 + omega_+*omega_-)*(omega_+ + omega_-)*(cos(2*omega_p*t + 2*phi_p) + 1)/((-omega_p + omega_+)*(-omega_p + omega_-)*(omega_p + omega_+)*(omega_p + omega_-))

### $a^2 \sigma_z$

In [14]:
# Term obtained in the paper
sp.expand_complex(-sp.Rational(1,2) * gt * ((sp.conjugate(xi_minus) + sp.conjugate(xi_plus)))).expand().trigsimp().subs({
    omega_q + omega_a : omega_plus,
    omega_q - omega_a : omega_minus,
    hbar:1
}).subs(omega_q**2 - omega_a**2, (omega_plus*omega_minus)).subs({
    omega_a : sp.Rational(1,2) * (omega_plus - omega_minus),
    omega_q : sp.Rational(1,2) * (omega_plus + omega_minus),
    }).expand().cancel().factor()

-|{g_p}|**2*(omega_+ + omega_-)*(-omega_p**2*cos(2*omega_p*t + 2*phi_p) - omega_p**2 - I*omega_p*omega_+*sin(2*omega_p*t + 2*phi_p) + I*omega_p*omega_-*sin(2*omega_p*t + 2*phi_p) + omega_+*omega_-*cos(2*omega_p*t + 2*phi_p) + omega_+*omega_-)/((-omega_p + omega_+)*(-omega_p + omega_-)*(omega_p + omega_+)*(omega_p + omega_-))

In [21]:
# Term obtained with SymPT
sp.expand_complex(sp.factor_terms(group_by_operators(H2)[sz*a**2]).expand().factor().simplify()).trigsimp().subs(hbar, 1).subs({
    omega_q + omega_a : omega_plus,
    omega_q - omega_a : omega_minus,
    hbar:1
}).subs(omega_q**2 - omega_a**2, (omega_plus*omega_minus)).subs({
    omega_a : sp.Rational(1,2) * (omega_plus - omega_minus),
    omega_q : sp.Rational(1,2) * (omega_plus + omega_minus),
    }).expand().cancel().factor()

-|{g_p}|**2*(omega_+ + omega_-)*(-omega_p**2*cos(2*omega_p*t + 2*phi_p) - omega_p**2 - I*omega_p*omega_+*sin(2*omega_p*t + 2*phi_p) + I*omega_p*omega_-*sin(2*omega_p*t + 2*phi_p) + omega_+*omega_-*cos(2*omega_p*t + 2*phi_p) + omega_+*omega_-)/((-omega_p + omega_+)*(-omega_p + omega_-)*(omega_p + omega_+)*(omega_p + omega_-))